In [3]:
import sys
import pickle
import torch 
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import random_split

# function to output the state space dynamics of the orbit
def pendfunc(t,x,u):
    g = 9.81  # gravity
    L = 5  # length     
    fdyn = np.array([x[1], -g*np.sin(x[0])/L+u[0]])
    return fdyn


# runge-kutta fourth-order numerical integration
def rk4(func, tk, _yk, uk, _dt, **kwargs):
    """
    single-step fourth-order numerical integration (RK4) method
    func: system of first order ODEs
    tk: current time step
    _yk: current state vector [y1, y2, y3, ...]
    _dt: discrete time step size
    **kwargs: additional parameters for ODE system
    returns: y evaluated at time k+1
    """

    # evaluate derivative at several stages within time interval
    f1 = func(tk, _yk,uk, **kwargs)
    f2 = func(tk + _dt / 2, _yk + (f1 * (_dt / 2)),uk, **kwargs)
    f3 = func(tk + _dt / 2, _yk + (f2 * (_dt / 2)),uk, **kwargs)
    f4 = func(tk + _dt, _yk + (f3 * _dt),uk, **kwargs)

    # return an average of the derivative over tk, tk + dt
    return np.array(_yk + (_dt / 6) * (f1 + (2 * f2) + (2 * f3) + f4))

device = torch.device("cuda:0")

number_of_initial_conditions = 15
dt = 0.01 # fidelity of solution
tf = 2 # length of time to generate set of data per initial condition
dp = int(tf/dt)  # number of data points
nx = 2 # state space representation of the model has 4 states
m  = 1 # number of control inputs 
predict_time = 25

# creating empty vectors to store data
data_x = torch.zeros((0,dp-predict_time+1, nx))
data_y = torch.zeros((0,dp-predict_time+1, nx))
data_y2 = torch.zeros((0,dp-predict_time+1, nx))
data_u = torch.zeros((0,dp-predict_time+1, m))

# looping through all the initial conditions
for num in range(number_of_initial_conditions):

    x_init = 2*torch.Tensor(nx).uniform_(-1, 1) # initial condition for the trajectory
    time = np.linspace(0, tf, dp) # simulate for given amount of time
    xk = x_init# + (0.001**0.5)*np.random.normal(0,1,2) # adding random Gaussian noise
    X = np.empty([nx,]) # setting up array to store current time step history
    Y = np.empty([nx,]) # setting up array to store next time step history
    usol = np.empty([m,]) # setting up array that holds control history

    # looping through the whole trajectory
    for t in time:
        
        uk = 0.05*torch.Tensor(m).uniform_(0, 1) # random control effort
        usol = np.vstack((usol, uk)) # stroring control effort
        X = np.vstack((X, xk)) # storing current time step 
        xk = rk4(pendfunc, t, xk, uk, dt)# + (0.001**0.5)*np.random.normal(0,1,2) # RK4 approximation of solution with added Gaussian noise
        Y = np.vstack((Y, xk)) # storing next time step

    X = torch.FloatTensor(X)
    Y = torch.FloatTensor(Y)
    usol = torch.FloatTensor(usol)

    # removing initialised column in array
    newx = X[1:-(predict_time-1),:]
    newy = Y[1:-(predict_time-1),:]
    usol = usol[1:-(predict_time-1),:]
    newy2 = Y[predict_time:,:]

    # storing in one large data array
    data_x = torch.vstack([data_x, newx[None,:]])
    data_y = torch.vstack([data_y, newy[None,:]])
    data_y2 = torch.vstack([data_y2, newy2[None,:]])
    data_u = torch.vstack([data_u, usol[None,:]])

dp = dp - (predict_time-1)
# saving data in files
with open('pendulum_variables_data.pkl', 'wb') as f:
    pickle.dump([tf,dp,dt,nx,m,predict_time], f)
torch.save(data_x,'data_x_c_noisy.pt')
torch.save(data_y,'data_y_c_noisy.pt')
torch.save(data_y2,'data_y2_c_noisy.pt')
torch.save(data_u,'data_u_c_noisy.pt')
